<a href="https://colab.research.google.com/github/danadler-dev/mesa_examples/blob/main/mesa_tutorial_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple model updating in a grid - based on [Mesa Tutorial](https://mesa.readthedocs.io/en/stable/tutorials/intro_tutorial.html)

In [ ]:
!pip install mesa

In [2]:
import numpy as np
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual, IntSlider
from matplotlib import animation, rc
from IPython.display import HTML
from operator import mod 

Model and Agent based on NxM grid. At each step the agents add an increment to their color

In [3]:
class ColorModel(Model):
  def __init__(self, X, Y):
    self.grid = SingleGrid(X, Y, True)
    self.schedule = RandomActivation(self)
    for x in range(X):
      for y in range(Y):
        a = ColorAgent(self, (x,y), (x/X, y/Y, 0))
        self.grid.place_agent(a, (x, y))
        self.schedule.add(a)
  def step(self):
    self.schedule.step()
      
class ColorAgent(Agent):
  def __init__(self, model, n, color):
    super().__init__(n, model)
    self.color = color
  def step(self):
    self.color = tuple((i+0.1)%1.0 for i in self.color)

Use the [@interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) decorator to define X, Y and steps. Then run the model forward that many steps - showing the image at the **end** of the run

In [4]:
@interact(X=(0, 500, 50), Y=(0, 500, 50), steps=(0, 100, 5))
def run(X=200, Y=200, steps=10):
  model = ColorModel(X, Y)
  im = np.zeros((model.grid.width, model.grid.height,3), dtype=float)
  fig, ax = plt.subplots(figsize=(X/50, Y/50))
  for i in range(steps):
    model.step()
    for agent, x, y in model.grid.coord_iter():
      im[x,y] = agent.color
  ax.imshow(im, aspect='auto')


interactive(children=(IntSlider(value=200, description='X', max=500, step=50), IntSlider(value=200, descriptio…

To show the image interactively during each step, we use [@interact_manual](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#interact_manual) to set the parameters and start the run. Then, we use [animation.FuncAnimation](https://colab.research.google.com/drive/1lnl5UPFWVPrryaZZgEzd0theI6S94c3X) to run the model forward and show the image at each step.

In [5]:
@interact_manual(X=(0, 500, 50), Y=(0, 500, 50), steps=(0, 100, 5))
def run(X=200, Y=200, steps=10):
  model = ColorModel(X, Y)
  im = np.zeros((model.grid.width, model.grid.height,3), dtype=float)
  fig, ax = plt.subplots(figsize=(X/50, Y/50))

  # animation function. This is called sequentially  
  def animate(i):
    model.step()
    for agent, x, y in model.grid.coord_iter():
      im[x,y] = agent.color
    ax.imshow(im, aspect='auto')
    return (fig,)
    
  anim = animation.FuncAnimation(fig, animate, frames=steps, interval=300, blit=True)
  plt.close()
  rc('animation', html='jshtml')
  return anim


interactive(children=(IntSlider(value=200, description='X', max=500, step=50), IntSlider(value=200, descriptio…